In [18]:
import cv2
import os

def trim_video(input_path, output_path, start_threshold=0.5, end_threshold=0.5, change_threshold=0.02):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error opening video file: {input_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_seconds = frame_count / fps

    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        print(f"Error reading video frame: {input_path}")
        cap.release()
        return

    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Find the frame where the significant change starts
    start_frame = None
    for frame_index in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, curr_frame = cap.read()
        if not ret:
            print(f"Error reading video frame: {input_path}")
            cap.release()
            return

        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the percentage of change between frames
        frame_diff = cv2.absdiff(curr_frame_gray, prev_frame_gray)
        frame_diff_percentage = cv2.countNonZero(frame_diff) / frame_diff.size

        if frame_diff_percentage >= change_threshold:
            start_frame = frame_index
            break

    if start_frame is None:
        print(f"No significant change detected in video: {input_path}")
        cap.release()
        return

    # Calculate the start and end timestamps for trimming
    start_timestamp = (start_frame / fps) - start_threshold

    # Find the last frame with significant change after the start frame
    end_frame = None
    for frame_index in range(start_frame, frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, curr_frame = cap.read()
        if not ret:
            print(f"Error reading video frame: {input_path}")
            cap.release()
            return

        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the percentage of change between frames
        frame_diff = cv2.absdiff(curr_frame_gray, prev_frame_gray)
        frame_diff_percentage = cv2.countNonZero(frame_diff) / frame_diff.size

        if frame_diff_percentage < change_threshold:
            end_frame = frame_index
            break

    # Calculate the end timestamp for trimming
    if end_frame is None:
        end_timestamp = total_seconds
    else:
        end_timestamp = (end_frame / fps) + end_threshold

    # Reset video capture and write trimmed video
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_timestamp * fps))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (prev_frame.shape[1], prev_frame.shape[0]), True)

    while cap.get(cv2.CAP_PROP_POS_FRAMES) / fps <= end_timestamp:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    # Release video capture and writer
    cap.release()
    out.release()


# Set the input and output folders
input_folder = r'C:\Users\edulo\OneDrive\Documentos\Loyalist\Term 2\Step Presentation - Project\SL Project\WLASL_videos\Input Trimming'
output_folder = r'C:\Users\edulo\OneDrive\Documentos\Loyalist\Term 2\Step Presentation - Project\SL Project\WLASL_videos\Output Trimming'

# Iterate over the input folder and trim each video
for filename in os.listdir(input_folder):
    if filename.endswith(".mp4"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        trim_video(input_path, output_path)

Error reading video frame: C:\Users\edulo\OneDrive\Documentos\Loyalist\Term 2\Step Presentation - Project\SL Project\WLASL_videos\Input Trimming\37006.mp4
